In [1]:
import numpy as np
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
# from tqdm import tqdm
import os 

import pdb

from trainer import *
from utils import *

/home/talaei/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def log(new_results, **kwargs):
    file_path = setup["file_path"]
    results = torch.load(file_path)
    print(len(list(new_results.keys())))
    results.update(new_results)
    print(len(results.keys()))
    torch.save(results, file_path)

In [43]:
def choose_kernel(IKM, iter_number, report, **kwargs):
    for kernel, params in setup[f"kernel {iter_number}"]:
        if kernel == "rbf":
            new_report = report + "-> rbf"
        elif kernel == "linear":
            new_report = report + "-> linear"
        for key, val in params.items():
            new_report += f", {key}: {str(val)}"
        if iter_number == 1:
            IKM.set_Ds()
            IKM.make_kernel_matrices(0, kernel, **params)
            cross_validate(IKM, iter_number, new_report, **kwargs)
        else:
            choose_g(IKM, iter_number, new_report, kernel, **kwargs)

In [32]:
def choose_g(IKM, iter_number, report, kernel, **kwargs):
    yhat, preds = kwargs.get("yhat"), kwargs.get("preds")
    for g in setup[f"g {iter_number}"]:
        if g == "identity":
            new_report = report + "-> g identity"
        elif g == "normalize":
            new_report = report + "-> g normalize"
            ## Normalize the predictions
            yhat, preds = normalize_preds(yhat, preds)
        IKM.set_Ds(yhat=yhat, preds=preds)
        IKM.make_kernel_matrices(ind=iter_number, kernel=kernel)
        mix_kernels(IKM, iter_number, new_report, **kwargs)

In [33]:
def mix_kernels(IKM, iter_number, report, **kwargs):
    if iter_number > 1:
        for weights in setup[f"mixing {iter_number}"]:
            new_report = report + f"-> mix ({', '.join(list(map(lambda x: str(x), weights)))})"
            IKM.combine_kernels(weights)
            cross_validate(IKM, iter_number, new_report, **kwargs)

In [38]:
def cross_validate(IKM, iter_number, report, **kwargs):
    IKM_copy = deepcopy(IKM)
    avg_diag = IKM.avg_diag_of_kernel()
    verbose_level = kwargs.get("verbose_level", 0)
    verbose = iter_number <= verbose_level
    for log_reg_ratio in setup[f"log_regs {iter_number}"]:
        reg = avg_diag * (10**log_reg_ratio)
        new_report = report + f"-> reg {reg:.3f}"
        # if new_report not in logs:
        if verbose:
            print(new_report)
        yhat, preds, res = IKM.solve(reg, verbose=verbose)
        if verbose:
            print(f"______________________________________")
        # log(new_report, res, **kwargs)
        results = kwargs["new_results"]
        results[new_report] = res
        kwargs["yhat"], kwargs["preds"] = yhat, preds
        IKM_copy = deepcopy(IKM)
        perform_iteration(IKM, iter_number+1, new_report, **kwargs)
        IKM = IKM_copy

In [39]:
def perform_iteration(IKM, iter_number, report, **kwargs):
    max_iter = setup["max iterations"]
    if iter_number <= max_iter:
        if iter_number <= kwargs.get("verbose_level", 0):
            print(f"------- level: {iter_number} -------")
        new_report = report + f" /{iter_number}: "
        if iter_number == 1:
            kwargs["new_results"] = {}
        choose_kernel(IKM, iter_number, new_report, **kwargs)
        if iter_number == 1:
            print(len(list(kwargs["new_results"].keys())))
            log(**kwargs)

In [44]:
dataset_name = "pointer_majority_3_3"
max_iter = 2
verbose_level=1
run_name = "rbf_var_ablation"
directory = f"./results/pvr/{dataset_name}/"
if not os.path.exists(directory):
        os.makedirs(directory) 
file_path = directory + run_name
if not os.path.exists(file_path):
    torch.save({}, file_path)
logs = torch.load(file_path)
setup = {"max iterations": max_iter,
 "file_path": file_path,
 "kernel 1": [ ("linear", {}), ("rbf", {'var':1}), ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})], #, ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})
 "kernel 2": [ ("linear", {}), ("rbf", {'var':1}), ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})],
 "kernel 3": ["linear", "rbf"],
 "kernel 4": ["linear", "rbf"],
 "kernel 5": ["linear", "rbf"],
 "g 2": ["normalize"],
 "g 3": ["normalize"],
 "g 4": ["normalize"],
 "mixing 2": [[0.75, 0.25], [0.50, 0.50], [0.25, 0.75], [0.00, 1.00]], 
 "mixing 3": [[0.34, 0.33, 0.33]], 
 "mixing 4": [[0.25, 0.25, 0.25, 0.25],], 
 "log_regs 1": [ -3, -2, -1.5, -1, 0, 1],  
 "log_regs 2": [ -3, -2, -1.5, -1, 0, 1],
 "log_regs 3": [ -3, -1.5, -1, 0, 1],
 "log_regs 4": [ -3, -1.5, -1, 0, 1],
 }
datasets = load_dataset(dataset_name)
IKM = IterativeKernelModel(dataset_name, datasets=datasets)
perform_iteration(IKM, 1, "", verbose_level=verbose_level)
title = f"pvr/{dataset_name}/{run_name}"
update_spread_sheet(title, file_path)
        

Train samples: 1638, Test samples: 410
------- level: 1 -------
 /1: -> linear-> reg 0.001
iteration took 0.034212 seconds
Training Error is 0.907106
Test Error is 0.923891
Training Accuracy is 0.621490
Test Accuracy is 0.604878
______________________________________
 /1: -> linear-> reg 0.010
iteration took 0.019278 seconds
Training Error is 12419991.067333
Test Error is 12534305.299763
Training Accuracy is 0.506716
Test Accuracy is 0.487805
______________________________________
 /1: -> linear-> reg 0.032
iteration took 0.020368 seconds
Training Error is 0.902772
Test Error is 0.925424
Training Accuracy is 0.625153
Test Accuracy is 0.612195
______________________________________
 /1: -> linear-> reg 0.100
iteration took 0.000564 seconds
Training Error is 0.902798
Test Error is 0.925400
Training Accuracy is 0.625763
Test Accuracy is 0.612195
______________________________________
 /1: -> linear-> reg 1.000
iteration took 0.000531 seconds
Training Error is 0.902815
Test Error is 0.9249

In [28]:
dataset_name = "pointer_3"
(X_train, y_train, X_test, y_test) = load_dataset(dataset_name)
model, optimizer = make_model([11, 10000, 1])
train_model(model, optimizer, X_train, y_train, epochs=10)
print("Train result:")
evaluate_model(model, X_train, y_train)
print("Test result:")
evaluate_model(model, X_test, y_test)

Train result:
- loss is 0.4868.
- acc is 85.4090%.
Test result:
- loss is 0.4979.
- acc is 81.4634%.


(tensor(0.4979), 81.46341463414633)

In [43]:
dataset_name = "pointer_majority_3_3"
(X_train, y_train, X_test, y_test) = load_dataset(dataset_name)
model, optimizer = make_model([11, 10000, 1])
train_model(model, optimizer, X_train, y_train, epochs=10)
print("Train result:")
evaluate_model(model, X_train, y_train)
print("Test result:")
evaluate_model(model, X_test, y_test)

Train result:
- loss is 0.4976.
- acc is 83.9438%.
Test result:
- loss is 0.4694.
- acc is 85.6098%.


(tensor(0.4694), 85.60975609756098)